In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA, ARMA

plt.rcParams['figure.figsize'] = (15.0, 5.0)

In [ ]:
df = pd.read_csv('../data/bitcoin_price.csv')

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.set_index('Date')
prices = df['Close']

plt.plot(prices)
plt.show()

In [ ]:
prices_log = pd.Series(np.log(prices)).asfreq('1d')
returns = (prices_log - prices_log.shift()).dropna()
returns_abs = pd.Series(np.abs(returns))
returns.plot()
plt.show()
result = adfuller(returns_abs, autolag='AIC')
output = pd.Series(result[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
output

In [ ]:
lag_acf = acf(returns_abs, nlags=20)
lag_pacf = pacf(returns_abs, nlags=20, method='ols')

######################### ACF ##########################################

plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--',color='blue')
plt.axhline(y=-1.96/np.sqrt(len(returns_abs)), linestyle='--', color='pink')
plt.axhline(y=1.96/np.sqrt(len(returns_abs)), linestyle='--', color='blue')
plt.title('Autocorrelation Function')

######################### PACF ##########################################

plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--',color='blue')
plt.axhline(y=-1.96/np.sqrt(len(returns_abs)), linestyle='--', color='pink')
plt.axhline(y=1.96/np.sqrt(len(returns_abs)), linestyle='--', color='blue')
plt.title('partial autocorrelation plot')
plt.show()

In [ ]:
## AR
plt.subplot(131)
model = ARMA(returns_abs, order=(1, 0))
result_AR = model.fit(disp=-1)
plt.plot(returns_abs)
plt.plot(result_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((result_AR.fittedvalues - returns)**2))

## MA
plt.subplot(132)
model = ARMA(returns_abs, order=(0, 1))
result_MA = model.fit(disp=-1)
plt.plot(returns_abs)
plt.plot(result_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((result_MA.fittedvalues-returns)**2))

## ARMA
plt.subplot(133)
model = ARMA(returns_abs, order=(1, 1))
result_ARMA = model.fit(disp=-1)
plt.plot(returns_abs)
plt.plot(result_ARMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((result_ARMA.fittedvalues-returns)**2))
plt.show()

In [ ]:
plt.plot(returns_abs)
plt.plot(result_ARMA.fittedvalues, color='red')
plt.show()